In [10]:
import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import Button, VBox, Output
from ipywidgets import Button, HBox, Output
import IPython.display as display

# csv path
csv_file_path = 'MostPopularShows.csv'

# Path to SQLite
sqlite_db_path = 'Netflix.db'

df = pd.read_csv(csv_file_path)

# connect to SQLite db
conn = sqlite3.connect(sqlite_db_path)

# convert data to SQLite table
df.to_sql('Netflix_Most_Popular_Shows', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

In [11]:
# Open connection to SQLite db
conn = sqlite3.connect(sqlite_db_path)

# Read SQLite info
df = pd.read_sql_query("SELECT * FROM Netflix_Most_Popular_Shows", conn)

# Close the connection
conn.close()

print(df)
df.info()
df.head()

     index                       TITLE  RELEASE_YEAR  SCORE  NUMBER_OF_VOTES  \
0        0                Breaking Bad          2008    9.5          1727694   
1        1  Avatar: The Last Airbender          2005    9.3           297336   
2        2                  Our Planet          2019    9.3            41386   
3        3                Kota Factory          2019    9.3            66985   
4        4              The Last Dance          2020    9.1           108321   
..     ...                         ...           ...    ...              ...   
241    241                 Evil Genius          2018    7.5            27516   
242    242              13 Reasons Why          2017    7.5           282373   
243    243                       Lupin          2021    7.5           100575   
244    244          All of Us Are Dead          2022    7.5            41393   
245    245     I Am Not Okay with This          2020    7.5            56459   

     DURATION  NUMBER_OF_SEASONS   MAIN

,index,TITLE,RELEASE_YEAR,SCORE,NUMBER_OF_VOTES,DURATION,NUMBER_OF_SEASONS,MAIN_GENRE,MAIN_PRODUCTION
0,0,Breaking Bad,2008,9.5,1727694,48,5,drama,US
1,1,Avatar: The Last Airbender,2005,9.3,297336,24,3,scifi,US
2,2,Our Planet,2019,9.3,41386,50,1,documentary,GB
3,3,Kota Factory,2019,9.3,66985,42,2,drama,IN
4,4,The Last Dance,2020,9.1,108321,50,1,documentary,US


In [12]:
genres = df['MAIN_GENRE'].unique()

# Function to create visualization
def topshows(genre):
    genredf = df[df['MAIN_GENRE'] == genre]
    top_shows = genredf.nlargest(20, 'SCORE')
    
    plt.figure(figsize=(12, 8))
    barplot = sns.barplot(x='SCORE', y='TITLE', data=top_shows)
    
    for p in barplot.patches:
        barplot.annotate(format(p.get_width(), '.1f'), 
                         (p.get_width(), p.get_y() + p.get_height() / 2),
                         ha = 'left', va = 'center', 
                         xytext = (4, 0), 
                         textcoords = 'offset points')

    plt.title(f'Top Netflix Shows In Each Genre (up to 20 shown): {genre}')
    plt.xlabel('Score')
    plt.ylabel('Title')
    plt.xlim(top_shows['SCORE'].min() - 0.1, top_shows['SCORE'].max() + 0.1)
    plt.show()

# Button function
def button_click(b):
    with output:
        output.clear_output()
        topshows(b.description)

buttons = [Button(description=genre) for genre in genres]
for button in buttons:
    button.on_click(button_click)

buttons_box = VBox(buttons)

# Display the widget output
output = Output()
display.display(buttons_box, output)

Output()